In [ ]:
# FINAL YEAR PROJECT - CODE FOR ANN

In [1]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

import optuna
import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense

from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Nadam

from sklearn.metrics import roc_auc_score

In [4]:
data = pd.read_csv(r'defaultforann.csv', encoding='latin1')

data

,LIMIT_BAL,AGE,default,Delay_Status,Delay_Count,Max_Delay,Delay_Category,Delay_Days,Sequantial_Delay_Status,Has_Early_Payment,...,Delay_Status_sum_by_Max_Delay,Delay_Status_min_by_Max_Delay,Delay_Status_max_by_Max_Delay,SEX_male,EDUCATION_high school,EDUCATION_not educated,EDUCATION_others,EDUCATION_university,MARRIAGE_others,MARRIAGE_single
0,20000,24,1,1,2,2,1,60,1,1,...,7187,1,1,0,0,0,0,1,0,0
1,120000,26,1,1,2,2,1,60,0,1,...,7187,1,1,0,0,0,0,1,0,1
2,90000,34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,50000,37,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,50000,57,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,39,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
29996,150000,43,0,0,0,0,0,0,0,1,...,0,0,0,1,1,0,0,0,0,1
29997,30000,37,1,1,3,4,2,90,1,1,...,218,1,1,1,0,0,0,1,0,1
29998,80000,41,1,1,1,1,1,30,0,1,...,1689,1,1,1,1,0,0,0,0,0


In [7]:
data.describe(include='all')

,LIMIT_BAL,AGE,default,Delay_Status,Delay_Count,Max_Delay,Delay_Category,Delay_Days,Sequantial_Delay_Status,Has_Early_Payment,...,Delay_Status_sum_by_Max_Delay,Delay_Status_min_by_Max_Delay,Delay_Status_max_by_Max_Delay,SEX_male,EDUCATION_high school,EDUCATION_not educated,EDUCATION_others,EDUCATION_university,MARRIAGE_others,MARRIAGE_single
count,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,...,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000,29000.000000
mean,167879.299310,35.435276,0.220379,0.335207,0.832069,0.435483,0.348241,24.962069,0.173448,0.463517,...,1839.528241,0.335207,0.335207,0.381828,0.162379,0.000483,0.014931,0.467655,0.012621,0.532172
std,129730.011561,9.185136,0.414510,0.472071,1.552802,1.340420,0.503038,46.584047,0.378641,0.498676,...,3027.618035,0.472071,0.472071,0.485843,0.368805,0.021967,0.121279,0.498961,0.111633,0.498972
min,10000.000000,21.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,140000.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,240000.000000,41.000000,0.000000,1.000000,1.000000,2.000000,1.000000,30.000000,0.000000,1.000000,...,1689.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1000000.000000,79.000000,1.000000,1.000000,6.000000,8.000000,2.000000,180.000000,1.000000,1.000000,...,7187.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
targets = data['default']

inputs = data.drop(['default'],axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(inputs)

scaled = scaler.transform(inputs)

inputs_scaled = pd.DataFrame(scaled, columns=inputs.columns)

inputs_scaled

,LIMIT_BAL,AGE,Delay_Status,Delay_Count,Max_Delay,Delay_Category,Delay_Days,Sequantial_Delay_Status,Has_Early_Payment,Avg_Bill_Amt,...,Delay_Status_sum_by_Max_Delay,Delay_Status_min_by_Max_Delay,Delay_Status_max_by_Max_Delay,SEX_male,EDUCATION_high school,EDUCATION_not educated,EDUCATION_others,EDUCATION_university,MARRIAGE_others,MARRIAGE_single
0,-1.139920,-1.244998,1.408273,0.752157,1.167204,1.295668,0.752157,2.182981,1.075833,-0.691045,...,1.766261,1.408273,1.408273,-0.785921,-0.440293,-0.021977,-0.123115,1.066924,-0.113058,-1.066555
1,-0.369075,-1.027251,1.408273,0.752157,1.167204,1.295668,0.752157,-0.458089,1.075833,-0.666336,...,1.766261,1.408273,1.408273,-0.785921,-0.440293,-0.021977,-0.123115,1.066924,-0.113058,0.937598
2,-0.600329,-0.156263,-0.710090,-0.535859,-0.324891,-0.692289,-0.535859,-0.458089,-0.929512,-0.443376,...,-0.607593,-0.710090,-0.710090,-0.785921,-0.440293,-0.021977,-0.123115,1.066924,-0.113058,0.937598
3,-0.908667,0.170357,-0.710090,-0.535859,-0.324891,-0.692289,-0.535859,-0.458089,-0.929512,-0.101511,...,-0.607593,-0.710090,-0.710090,-0.785921,-0.440293,-0.021977,-0.123115,1.066924,-0.113058,-1.066555
4,-0.908667,2.347825,-0.710090,-0.535859,-0.324891,-0.692289,-0.535859,-0.458089,1.075833,-0.423115,...,-0.607593,-0.710090,-0.710090,1.272393,-0.440293,-0.021977,-0.123115,1.066924,-0.113058,-1.066555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28995,-0.523244,0.496977,1.408273,2.040174,1.167204,1.295668,2.040174,2.182981,-0.929512,-0.006291,...,1.766261,1.408273,1.408273,-0.785921,-0.440293,-0.021977,-0.123115,1.066924,-0.113058,0.937598
28996,1.018446,0.496977,-0.710090,-0.535859,-1.070938,-0.692289,-0.535859,-0.458089,1.075833,-0.705186,...,-0.607593,-0.710090,-0.710090,-0.785921,2.271216,-0.021977,-0.123115,-0.937274,-0.113058,0.937598
28997,2.174713,0.496977,-0.710090,-0.535859,-0.324891,-0.692289,-0.535859,-0.458089,1.075833,-0.349549,...,-0.607593,-0.710090,-0.710090,-0.785921,2.271216,-0.021977,-0.123115,-0.937274,-0.113058,0.937598
28998,1.480953,0.823597,1.408273,0.108149,0.421157,1.295668,0.108149,-0.458089,1.075833,-0.711355,...,-0.049719,1.408273,1.408273,-0.785921,-0.440293,-0.021977,-0.123115,-0.937274,-0.113058,-1.066555


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=42)

In [11]:
def create_model(trial):

    model = Sequential()

    model.add(Dense(units=trial.suggest_int('units_layer1', 6, 32), activation='relu'))
    model.add(Dense(units=trial.suggest_int('units_layer2', 6, 32), activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))

    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop', 'adagrad'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'adagrad':
        optimizer = Adagrad(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])

    return model

In [12]:
def optimal(trial):

    epochs = trial.suggest_int('epochs', 10, 50)
    batch_size = trial.suggest_int('batch_size', 16, 64)

    model = create_model(trial)

    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)

    return auc

study = optuna.create_study(direction='maximize')
study.optimize(optimal, n_trials=10)

print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2025-01-20 04:51:57,259] A new study created in memory with name: no-name-f41aa350-59ee-476a-acd6-2f3c6668239e


Epoch 1/32


<ipython-input-11-f5e32df2cda7>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - AUC: 0.4904 - loss: 0.6111
Epoch 2/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - AUC: 0.5310 - loss: 0.5878
Epoch 3/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.5754 - loss: 0.5600
Epoch 4/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.6046 - loss: 0.5440
Epoch 5/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.6319 - loss: 0.5321
Epoch 6/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.6592 - loss: 0.5161
Epoch 7/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6791 - loss: 0.5033
Epoch 8/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.6772 - loss: 0.5031
Epoch 9/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.6973 - loss: 0.4874
Epoch 10/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.7037 - loss: 0.4788
Epoch 11/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.7084 - loss: 0.4783
Epoch 12/32
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.7076 - loss: 0.4768
Epoch 13/32
363/363 ━━━━━━━━━━━━━━━━

[I 2025-01-20 04:52:23,502] Trial 0 finished with value: 0.7313231222572757 and parameters: {'epochs': 32, 'batch_size': 64, 'units_layer1': 8, 'units_layer2': 8, 'optimizer': 'sgd', 'learning_rate': 0.0007397514640589673}. Best is trial 0 with value: 0.7313231222572757.


...

Epoch 1/43


<ipython-input-11-f5e32df2cda7>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


749/749 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - AUC: 0.4492 - loss: 0.6374
Epoch 2/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.4928 - loss: 0.6119
Epoch 3/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.5172 - loss: 0.5981
Epoch 4/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.5471 - loss: 0.5840
Epoch 5/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.5661 - loss: 0.5714
Epoch 6/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.5829 - loss: 0.5616
Epoch 7/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - AUC: 0.6021 - loss: 0.5535
Epoch 8/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - AUC: 0.6189 - loss: 0.5434
Epoch 9/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - AUC: 0.6255 - loss: 0.5382
Epoch 10/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6395 - loss: 0.5262
Epoch 11/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.6391 - loss: 0.5257
Epoch 12/43
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6570 - loss: 0.5189
Epoch 13/43
749/749 ━━━━━━━━━━━━━━━━

[I 2025-01-20 04:58:34,635] Trial 9 finished with value: 0.7158950423553042 and parameters: {'epochs': 43, 'batch_size': 31, 'units_layer1': 7, 'units_layer2': 13, 'optimizer': 'adagrad', 'learning_rate': 0.0004536689917977675}. Best is trial 1 with value: 0.7618921317926773.


Best trial: 0.7618921317926773
Best hyperparameters: {'epochs': 48, 'batch_size': 58, 'units_layer1': 10, 'units_layer2': 27, 'optimizer': 'adam', 'learning_rate': 0.0014019152273269052}


In [13]:
best_params = study.best_trial.params

best_params

{'epochs': 48,
 'batch_size': 58,
 'units_layer1': 10,
 'units_layer2': 27,
 'optimizer': 'adam',
 'learning_rate': 0.0014019152273269052}

In [14]:
best_model = Sequential()
best_model.add(Dense(units=best_params['units_layer1'], activation='relu'))
best_model.add(Dense(units=best_params['units_layer2'], activation='relu'))
best_model.add(Dense(1, activation='sigmoid'))

In [15]:
if best_params['optimizer'] == 'adam':
    best_optimizer = Adam(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'sgd':
    best_optimizer = SGD(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'rmsprop':
    best_optimizer = RMSprop(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'adagrad':
    best_optimizer = Adagrad(learning_rate=best_params['learning_rate'])


In [16]:
best_model.compile(optimizer=best_optimizer, loss='binary_crossentropy', metrics=['AUC'])

In [17]:
def evaluate(model, X_train, y_train, X_test, y_test):

    model.fit(X_train, y_train, epochs=10, batch_size=best_params['batch_size'])

    y_train_prob = model.predict(X_train)


    y_test_prob = model.predict(X_test)

    roc_train_prob = roc_auc_score(y_train, y_train_prob)
    gini_train_prob = roc_train_prob * 2 - 1


    roc_test_prob = roc_auc_score(y_test, y_test_prob)
    gini_test_prob = roc_test_prob * 2 - 1


    results = pd.DataFrame({
        'Dataset': ['Train', 'Test'],
        'Gini': [gini_train_prob * 100, gini_test_prob * 100],

    })

    return results

In [18]:
evaluate(best_model, X_train, y_train, X_test, y_test)

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - AUC: 0.6947 - loss: 0.4899
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7662 - loss: 0.4440
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7728 - loss: 0.4400
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7592 - loss: 0.4466
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7680 - loss: 0.4368
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7785 - loss: 0.4292
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7696 - loss: 0.4384
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7745 - loss: 0.4367
Epoch 9/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7767 - loss: 0.4332
Epoch 10/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7737 - loss: 0.4368
725/725 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


,Dataset,Gini
0,Train,55.409046
1,Test,52.726947
